In [1]:
%load_ext autoreload
%autoreload 2

# DeepSeek on VKS documents

## Using PostgreSQL to retrieval VKS documents


Define global variables

In [2]:
import os

from langchain_community.llms import VLLMOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

from modules import utils

from IPython.display import display, Markdown, Latex

In [3]:
ENV_FILE_PATH = "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/env"
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
MAX_TOKENS = 32700
COLLECTION_NAME = "my_docs"

Load environment variables

In [4]:
envs = utils.load_env_to_dict(ENV_FILE_PATH)

## Prepare PostgreSQL vectorstore

In [5]:
from langchain_postgres.vectorstores import PGVector
from langchain_huggingface.embeddings import HuggingFaceEmbeddings


In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
)

/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
vector_store = PGVector(
    embeddings=embeddings,
    collection_name=COLLECTION_NAME,
    connection=envs["POSTGRESQL_URI_2"],
    use_jsonb=True,
)

## Retrieval

In [8]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [9]:
res = vector_store.similarity_search("VKS là gì?")
utils.print_documents(res)

Amount of documents: 4
Document 0:
VKS là gì?

VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các ứng dụng container trên môi trường phân tán.

Những điểm nổi bật của VKS

Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng bạn khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.

Hỗ trợ các phiên bản Kubernetes mới nhất: VKS luôn cập nhật những phiên bản Kubernetes mới nhất (minor version từ 1.27, 1.28, 1.29) để đảm bảo bạn luôn tận dụng được những tính năng tiên tiến nhất.

Kubernetes Networking: VKS tích hợp Calico CNI, mang lại tính hiệu quả và bảo mật cao.

Upgrade seamlessly: VKS hỗ trợ nâng cấp giữa các phiên bản Kubernetes một cách dễ dàng và nhanh chóng, giúp bạn luô

## Integrated with DeepSeek R1

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [11]:
# Connect to vLLM API
llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=envs['VLLM_HOST_URL_2'] + "/v1/",
    model_name=MODEL_NAME,
    max_tokens=MAX_TOKENS,
    streaming=True,
)

prompt = ChatPromptTemplate.from_template("""
Bạn là một trợ lí ảo nhằm trả lời các CÂU HỎI cho khách hàng của VNGCloud.
Câu trả lời của bạn phải có liên quan đến NGỮ CẢNH được cung cấp. Nếu không liên quan hãy yêu cầu cung cấp thêm thông tin.
Bạn PHẢI trả lời bằng TIẾNG VIỆT

CÂU HỎI: {query}

NGỮ CẢNH:
{context}

TRẢ LỜI:
""")

chain = prompt | llm

## Prepare the context

In [12]:
query = "VKS là gì?"
context = utils.from_documents_to_context(vector_store.similarity_search(query))

display(Markdown(context[:1000]))

- Document 1:
VKS là gì?

VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các ứng dụng container trên môi trường phân tán.

Những điểm nổi bật của VKS

Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng bạn khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.

Hỗ trợ các phiên bản Kubernetes mới nhất: VKS luôn cập nhật những phiên bản Kubernetes mới nhất (minor version từ 1.27, 1.28, 1.29) để đảm bảo bạn luôn tận dụng được những tính năng tiên tiến nhất.

Kubernetes Networking: VKS tích hợp Calico CNI, mang lại tính hiệu quả và bảo mật cao.

Upgrade seamlessly: VKS hỗ trợ nâng cấp giữa các phiên bản Kubernetes một cách dễ dàng và nhanh chóng, giúp bạn luôn cập nhật những cải 

In [13]:
query = "VKS là gì?"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được yêu cầu trả lời câu hỏi "VKS là gì?" tôi cần tìm hiểu thông tin từ các tài liệu cung cấp. Tài liệu 1 giải thích rõ VKS là dịch vụ Kubernetes được quản lý của VNGCloud, giúp đơn giản hóa việc triển khai và quản lý ứng dụng container. It nhấn mạnh các tính năng như quản lý tự động, hỗ trợ các phiên bản mới, tích hợp Calico CNI, và các tùy chọn mở rộng tự động.

Tài liệu 2 và 3 cũng định nghĩa VKS giống như Document 1, nhưng thêm các cập nhật tính năng mới như giám sát tài nguyên, tích hợp Terraform, cải tiến các plugin và chức năng mới như stop POC. Tài liệu 4 đi sâu vào cách sử dụng Terraform với VKS, hướng dẫn các bước tạo cluster và node group, cung cấp ví dụ cụ thể.

Từ các thông tin này, tôi hiểu VKS không chỉ là một dịch vụ Kubernetes thông thường mà còn được tích hợp nhiều tính năng hiện đại, hỗ trợ quản lý kolay dàng và tối ưu hóa chi phí. Vì vậy, trả lời cần bao gồm định nghĩa, các điểm nổi bật và maybe một số cập nhật mới để khách hàng hình dung đầy đủ về dịch 

In [14]:
query = "Làm sao để expose service trong VKS?"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Để expose một service trong VKS, bạn cần tạo một Kubernetes Service và expose nó thông qua một số phương pháp như ClusterIP, NodePort, LoadBalancer hoặc Ingress. Dưới đây là các bước tổng quát để expose service:

1. **Tạo Kubernetes Service:**
   - Sử dụng file định nghĩa YAML để tạo Service. Ví dụ:

     ```yaml
     apiVersion: v1
     kind: Service
     metadata:
       name: my-service
     spec:
       selector:
         app: my-app
       ports:
       - name: http
         port: 80
         targetPort: 8080
       type: LoadBalancer
     ```

2. **Áp dụng cấu hình:**
   - Chạy lệnh `kubectl apply -f service.yaml` để tạo Service.

3. **Kiểm tra trạng thái:**
   - Sử dụng `kubectl get svc` để xem thông tin Service và địa chỉ IP hoặc DNS được gán.

4. **Truy cập Service:**
   - Tùy thuộc vào loại Service:
     - **LoadBalancer:** Truy cập qua địa chỉ IP 外部 được cung cấp.
     - **NodePort:** Truy cập qua cổng được chỉ định trên tất cả các Node.
     - **Ingress:** Cài đ

In [15]:
query = "Hoàng Sa và Trường Sa là của ai?"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được rồi, systems cần phân tích câu hỏi từ người dùng và ngôn từ trả lời phù hợp. Khách hàng hỏi về chủ quyền Hoàng Sa và Trường Sa, một vấn đề nhạy cảm và quan trọng ở Việt Nam. Tuy nhiên, các document cung cấp đều liên quan đến công nghệ và Kubernetes, không hề nhắc đến địa lý hay chính trị. Vì vậy, tôi không thể đưa ra câu trả lời có liên quan dựa trên các thông tin đã có.월부터 phải yêu cầu khách hàng cung cấp thêm thông tin hoặc chuyển hướng câu hỏi sang chủ đề liên quan đến kỹ thuật nếu có thể.



GENERATED RESPONSE


Câu hỏi của bạn không liên quan đến các tài liệu được cung cấp. Vui lòng cung cấp thêm thông tin hoặc hỏi về các chủ đề kỹ thuật liên quan đến VNGCloud nếu bạn cần hỗ trợ.

In [16]:
query = "Tạo cho tôi PVC 20Gb trong vks"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được yêu cầu tạo PVC 20GB trong VKS. Đầu tiên xem xét các tài liệuêtre cung cấp.

Trong Document 1, có thông tin về việc nâng cấp PVC, cho phép kích thước tối thiểu là 1GB. Điều này chứng tỏ PVC 20GB là khả thi.

Document 2 và Document 4 hướng dẫn cách tạo PVC thông qua file YAML, bao gồm việc thêm tham số isPOC và cách cấu trúc file YAML.

Từ đó, tôi biết cần tạo file YAML với storage class và kích thước 20GB, đồng thời thêm tham số isPOC: "true" nếu cần sử dụng ví POC.

Kết hợp các thông tin, soạn thảo file YAML phù hợp và hướng dẫn khách hàng áp dụng.



GENERATED RESPONSE


Để tạo Persistent Volume Claim (PVC) 20GB trong VNGCloud Kubernetes Service (VKS), bạn có thể thực hiện theo các bước sau đây:

### 1. Tạo file YAML cho PVC
Tạo một file có tên `pvc-20gb.yaml` với nội dung như sau:

```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: pvc-20gb
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 20Gi
  storageClassName: m

In [17]:
query = "Làm sao để cấu hình volume type khi dùng pvc trong vks"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được rồi, tôi sắp xử lý câu hỏi của khách hàng về việc cấu hình volume type khi dùng PVC trong VKS. Đầu tiên, tôi cần xem qua các Document mà khách hàng đã cung cấp để tìm thông tin liên quan.

Trong Document 1, tôi thấy có một ví dụ về việc tạo PVC và Pod. Ở phần spec của PVC, có trường storageClassName được chỉ định là "my-expansion-storage-class". Điều này ám chỉ rằng để cấu hình volume type, chúng ta cần định nghĩa StorageClass phù hợp.

Document 4 nhắc đến việc thay đổi mặc định Storage Class từ IOPS 200 sang 3000. Điều này cho thấy rằng StorageClass được cấu hình để指定 volume type dựa trên IOPS hoặc các thông số khác.

Từ Document 3, tôi thấy có một phần về việc thêm annotation vào PVC để chỉ định volume type ID. Điều này cho thấy rằng ngoài việc định nghĩa StorageClass, việc thêm annotation có thể là một cách để chỉ định volume type cụ thể.

Vậy, để cấu hình volume type khi dùng PVC trong VKS, cần làm hai việc:

1. Tạo hoặc chọn StorageClass phù hợp với volume type mo